<a href="https://colab.research.google.com/github/HwangHanJae/Recommendation_Study/blob/main/inflearn_recsys/CF_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 협업 필터링

어떤 아이템에 대해 **비슷한 취향**을 가진 사람들은 다른 아이템 또한 **비슷한 취향**을 가질 것이다.

- 추천대상의 유사집단(피어그룹)을 통하여 추천대상에게 아이템을 추천하는 방법

피어그룹(유사집단)을 찾기 위하여 유사도를 계산
  - 유사도 : 피어슨 상관계수, 코사인 유사도



## 유사도 지표

피어슨 상관계수
- -1 ~ 1 사이의 값을 가짐


코사인 유사도
- 협업 필터링에서 가장 널리 쓰이는 유사도
- -1 ~ 1 사의 값
- 데이터가 이진값(binary)형태라면 타니모토 계수(tanimoto coefficient) 사용 권장

자카드 계수
- 타이모토 계수의 변형 
- 이진 데이터의 좋은 결과




## 과정
1. 모든 사용자간 평가의 유사도 계산
2. 추천 대상과 다른 사용자간 유사도 추출
3. 추천대상이 아직 평가하지 않은 아이템에 대하여 예상 평가값을 계산
- 평가값 : 다른 사용자 평가 * 다른 사용자 유사도
4. 아이템 중에서 예상 평가값 가장 높은 N개 추천

# 데이터 읽기

무비렌즈의 유저의 정보(u.user) 읽기

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#베이스 경로 설정
base = '/content/drive/MyDrive/RecoSys/Data'

# u.user 파일 경로 설정
u_user_path = os.path.join(base, 'u.user')

#필요한 컬럼 정의
u_cols = ['user_id','age','sex','occupation','zip_code']

#데이터 읽어오기
users = pd.read_csv(u_user_path, sep='|', names = u_cols, encoding='latin-1')
#users 데이터 프레임에 인덱스(user_id) 지정
users = users.set_index('user_id')

#상위 5개
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


무비렌즈의 영화의 정보(u.item) 읽기

In [2]:
#u.item의 파일 경로 설정
u_item_path = os.path.join(base, 'u.item')

#필요한 컬럼 정의
i_cols = ['movie_id','title','release date','video release date','IMDB URL','unknown','Action',
          'Adventure','Animation','Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama',
          'Fantasy','Film-Noir','Horror','Musical', 'Mystery','Romance','Sci-Fi','Thriller','War','Western']

# 데이터 읽어오기
movies = pd.read_csv(u_item_path, sep='|',names =i_cols, encoding='latin-1')
# movies 데이터 프레임에 인덱스(movie_id) 지정
movies = movies.set_index('movie_id')

#상위 5개
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


무비렌즈의 평점 정보(u.data)읽기

In [3]:
#u.data의 파일경로 지정
u_data_path = os.path.join(base, 'u.data')

#필요한 컬럼 정의
r_cols = ['user_id', 'movie_id','rating','timestamp']

#데이터 읽어오기
ratings = pd.read_csv(u_data_path, sep='\t',names = r_cols, encoding='latin-1')

#상위 5개
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# 평가 지표

In [32]:
from sklearn.metrics import mean_squared_error
def RMSE(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

In [33]:
def score(model):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])

  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])

  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

In [34]:
# 데이터 셋 만들기
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, stratify=y)

#sparse matrix 만들기
ratings_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [35]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = ratings_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)


In [36]:
# 주어진 영화(movie_id) 가중평균 rating을 계산하는 함수
def CF_simple(user_id, movie_id):
  #train set안에 movie_id가 존재한다면
  if movie_id in ratings_matrix.columns:
    #해당 유저(user_id)와 다른 유저의 유사도를 구함
    sim_scores = user_similarity[user_id].copy()
    #주어진 영화(movie_id)의 평점정보를 가져옴
    #어떤 유저가 평점을 했는지 안하였는지 확인할 수 있음
    movie_ratings = ratings_matrix[movie_id].copy()
    #주어진 영화(movie_id)에 대하여 아직 평가를 하지 않은 유저정보(user_id)를 가져옴
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index

    #평가를 하지 않은 유저를 movie_ratings와 sim_scores에서 drop시킴
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)

    #유사도와 영화의 평점을 가중평균하여 계산
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  else:
    mean_rating = 3.0

  return mean_rating

In [37]:
score(CF_simple)

1.0173831295637925

이전에 실행했던 1.03의 결과보다는 성능 개선의 효과를 볼 수 있었음